In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
from tensorflow.keras.models import load_model

EMOTION RECOGNITION MODEL - FER2013

DESCRIPCIÓN GENERAL
Este notebook entrena un modelo de Red Neuronal Convolucional (CNN) para reconocer emociones en imágenes faciales usando el dataset FER2013. El modelo es capaz de clasificar imágenes en 7 categorías de emociones diferentes.

FUNCIONALIDADES PRINCIPALES

1. CARGA Y PREPARACIÓN DE DATOS
- Carga de imágenes desde directorio de entrenamiento y prueba
- Aplicación de Data Augmentation en el conjunto de entrenamiento
- Normalización de imágenes (escala 0-1)
- Generadores de datos para procesamiento eficiente en lotes

2. ARQUITECTURA DEL MODELO CNN
- 4 bloques convolucionales con capas de convolución, normalización de lotes y dropout
- Pooling progresivo para reducción de dimensionalidad
- Capas densas (fully connected) para clasificación final
- Softmax para salida probabilística de 7 clases

3. ENTRENAMIENTO
- Optimizador: Adam (learning rate: 0.0001)
- Loss: Categorical Crossentropy
- Callbacks incluyen:
  - Early Stopping (paciencia: 10 epochs)
  - Reducción de tasa de aprendizaje (ReduceLROnPlateau)
  - Guardado del mejor modelo (ModelCheckpoint)

4. EMOCIONES DETECTADAS
1. Angry (Cabreado)
2. Disgust (Asqueado)
3. Fear (Asustado)
4. Happy (Feliz)
5. Neutral (Neutral)
6. Sad (Triste)
7. Surprise (Sorprendido)

5. EVALUACIÓN Y VISUALIZACIÓN
- Matriz de confusión
- Reporte de clasificación (precisión, recall, f1-score)
- Gráficos de accuracy y loss durante el entrenamiento
- Función de predicción para imágenes individuales

6. OUTPUTS
- best_emotion_model.h5: Mejor modelo guardado durante el entrenamiento
- final_emotion_model.h5: Modelo final después de todos los epochs
- training_history.png: Gráficos de entrenamiento
- confusion_matrix.png: Matriz de confusión del modelo

In [ ]:

# Configuración de rutas - ajusta según tu estructura en Kaggle
TRAIN_DIR = '/kaggle/input/fer2013/train'
TEST_DIR = '/kaggle/input/fer2013/test'
IMG_SIZE = 48
BATCH_SIZE = 64
EPOCHS = 50

# Clases de emociones
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
num_classes = len(emotions)

# Data Augmentation para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Solo normalización para validación/test
test_datagen = ImageDataGenerator(rescale=1./255)

# Generadores de datos
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

print(f"Clases encontradas: {train_generator.class_indices}")
print(f"Imágenes de entrenamiento: {train_generator.samples}")
print(f"Imágenes de prueba: {test_generator.samples}")

# Construcción del modelo CNN
def create_emotion_cnn():
    model = keras.Sequential([
        # Bloque 1
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 2
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 3
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 4
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Capas densas
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Crear y compilar el modelo
model = create_emotion_cnn()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_emotion_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=callbacks,
    verbose=1
)

# Guardar el modelo final
model.save('final_emotion_model.h5')
print("Modelo guardado exitosamente!")

# Visualizar resultados del entrenamiento
plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Evaluación del modelo
print("\n=== Evaluación en el conjunto de prueba ===")
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predicciones para la matriz de confusión
test_generator.reset()
predictions = model.predict(test_generator, steps=len(test_generator))
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Reporte de clasificación
print("\n=== Reporte de Clasificación ===")
print(classification_report(y_true, y_pred, target_names=emotions))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=emotions, yticklabels=emotions)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Función para predecir emociones en imágenes individuales
def predict_emotion(image_path):
    """
    Predice la emoción de una imagen individual
    """
    img = keras.preprocessing.image.load_img(
        image_path, 
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='grayscale'
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array, verbose=0)
    emotion_idx = np.argmax(prediction)
    confidence = prediction[0][emotion_idx]
    
    return emotions[emotion_idx], confidence

# Ejemplo de uso de la función de predicción
# emotion, conf = predict_emotion('path/to/image.jpg')
# print(f"Emoción detectada: {emotion} (Confianza: {conf:.2%})")

print("\n¡Entrenamiento completado exitosamente!")

: 

In [8]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import threading
import time
import os

# Cargar el modelo entrenado
model = load_model('best_emotion_model.h5')

# Emociones
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
emotions_spanish = ['Cabreado', 'Asqueado', 'Asustado', 'Feliz', 'Neutral', 'Triste', 'Sorprendido']

# Archivos PNG (ajusta nombres/rutas si es necesario)
png_files = {
    0: 'angry.jpg',    # angry -> Cabreado
    1: 'disgust.png',   # disgust -> Berto serio
    2: 'fear.png',      # fear -> Asustado
    3: 'happy.png',   # happy -> moguefeliz
    4: 'neutral.png',    # neutral -> Neutral
    5: 'sadness.png',       # sad -> Triste
    6: 'surprise.png'  # surprise -> Sorprendido
}

# Cargar las imágenes PNG con canal alfa (si existen)
png_images = {}
for idx, fname in png_files.items():
    if os.path.exists(fname):
        img = cv2.imread(fname, cv2.IMREAD_UNCHANGED)  # mantiene alfa
        if img is None:
            print(f"Advertencia: no se pudo leer {fname}")
        else:
            png_images[idx] = img
    else:
        print(f"Advertencia: archivo PNG no encontrado: {fname}")

# Definir colores y efectos por emoción (BGR)
emotion_config = {
    0: {'color': (0, 0, 255), 'name': 'Cabreado'},      # Rojo
    1: {'color': (0, 165, 255), 'name': 'Asqueado'},    # Naranja
    2: {'color': (255, 0, 0), 'name': 'Asustado'},      # Azul
    3: {'color': (0, 255, 0), 'name': 'Feliz'},         # Verde
    4: {'color': (128, 128, 128), 'name': 'Neutral'},   # Gris
    5: {'color': (255, 0, 255), 'name': 'Triste'},      # Magenta
    6: {'color': (0, 255, 255), 'name': 'Sorprendido'}  # Amarillo
}

# Cargar el cascada de clasificador para detección de rostros
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

# Variable de control
stop_event = threading.Event()


def overlay_png_on_frame(frame, png, x, y, w, h, scale=1.0):
    """Redimensiona y superpone `png` sobre `frame` centrado en la cara (x,y,w,h).
       `png` puede tener 3 o 4 canales (BGRA)."""
    if png is None:
        return frame

    # Calcular tamaño objetivo (un poco más grande que la cara)
    target_w = int(w * scale)
    target_h = int(h * scale)

    # Redimensionar la PNG manteniendo proporción
    png_h, png_w = png.shape[:2]
    if png_w == 0 or png_h == 0:
        return frame

    resized = cv2.resize(png, (target_w, target_h), interpolation=cv2.INTER_AREA)

    # Coordenadas para centrar la imagen sobre la cara
    cx = x + w // 2
    cy = y + h // 2
    x1 = int(cx - target_w // 2)
    y1 = int(cy - target_h // 2)
    x2 = x1 + target_w
    y2 = y1 + target_h

    # Recortar coordenadas que salgan del frame
    fh, fw = frame.shape[:2]
    ox1, oy1 = max(0, x1), max(0, y1)
    ox2, oy2 = min(fw, x2), min(fh, y2)

    if ox1 >= ox2 or oy1 >= oy2:
        return frame

    # Coordenadas relativas dentro de la imagen redimensionada
    rx1 = ox1 - x1
    ry1 = oy1 - y1
    rx2 = rx1 + (ox2 - ox1)
    ry2 = ry1 + (oy2 - oy1)

    # Extraer regiones
    roi_frame = frame[oy1:oy2, ox1:ox2]
    roi_png = resized[ry1:ry2, rx1:rx2]

    # Si la PNG tiene canal alfa, usarlo para mezclar
    if roi_png.shape[2] == 4:
        b, g, r, a = cv2.split(roi_png)
        alpha = a.astype(float) / 255.0
        alpha = cv2.merge([alpha, alpha, alpha])
        foreground = cv2.merge([b, g, r]).astype(float)
        background = roi_frame.astype(float)
        blended = cv2.convertScaleAbs(foreground * alpha + background * (1 - alpha))
    else:
        # PNG sin alfa: reemplazar directamente (o usar transparencia fija)
        blended = roi_png[:, :, :3]

    frame[oy1:oy2, ox1:ox2] = blended
    return frame


def add_emotion_effects(frame, emotion_idx, x, y, w, h, confidence):
    """Agrega efectos visuales según la emoción. Si existe una PNG para la emoción,
       la superpone sobre la cara; en caso contrario, dibuja los efectos previos."""
    config = emotion_config.get(emotion_idx, {'color': (255, 255, 255), 'name': str(emotion_idx)})

    # Si tenemos una PNG para esta emoción, superponerla (con un poco de escala)
    png = png_images.get(emotion_idx)
    if png is not None:
        # Escalas recomendadas: caras pequeñas -> ampliar un poco
        scale = 1.2
        frame = overlay_png_on_frame(frame, png, x, y, w, h, scale=scale)

        # También dibujar texto con nombre y confianza
        text = f"{config['name']}: {confidence:.2%}"
        cv2.putText(frame, text, (x + 5, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        return frame

    # Fallback: efectos originales si no hay PNG
    color = config['color']

    # 1. Cambiar el color del borde del rectángulo
    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

    # 2. Agregar fondo de color en la esquina superior
    overlay = frame.copy()
    cv2.rectangle(overlay, (x - 5, y - 50), (x + 200, y - 10), color, -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)

    # 3. Mostrar texto con emoción
    text = f"{config['name']}: {confidence:.2%}"
    cv2.putText(frame, text, (x + 5, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # 4. Agregar círculos decorativos alrededor del rostro
    cv2.circle(frame, (x + w//2, y + h//2), w//2 + 10, color, 2)

    # 5. Agregar efectos de partículas en las esquinas del rostro
    for corner_x, corner_y in [(x, y), (x+w, y), (x, y+h), (x+w, y+h)]:
        cv2.circle(frame, (corner_x, corner_y), 5, color, -1)

    return frame


def emotion_detection_thread():
    """Función que ejecuta la detección de emociones en un hilo separado"""

    # Inicializar la webcam
    cap = cv2.VideoCapture(0)

    # Configurar resolución
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    print("Ventana abierta. Presiona 'q' para salir.")
    print("Las emociones cambiarán el color y los efectos visuales\n")

    while not stop_event.is_set():
        ret, frame = cap.read()

        if not ret:
            print("Error al capturar la cámara")
            break

        # Convertir a escala de grises para detección de rostros
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar rostros
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.3,
            minNeighbors=5,
            minSize=(30, 30)
        )

        # Procesar cada rostro detectado
        emotion_detected = None
        max_confidence = 0

        for (x, y, w, h) in faces:
            # Extraer la región del rostro en escala de grises
            roi_gray = gray[y:y + h, x:x + w]

            # Redimensionar a 48x48 (tamaño del modelo)
            roi_resized = cv2.resize(roi_gray, (48, 48))

            # Normalizar
            roi_normalized = roi_resized / 255.0

            # Agregar dimensión de canal
            roi_input = np.expand_dims(roi_normalized, axis=-1)
            roi_input = np.expand_dims(roi_input, axis=0)

            # Realizar predicción
            prediction = model.predict(roi_input, verbose=0)
            emotion_idx = np.argmax(prediction)
            confidence = prediction[0][emotion_idx]

            # Guardar la emoción más confiable
            if confidence > max_confidence:
                max_confidence = confidence
                emotion_detected = emotion_idx

            # Agregar efectos según la emoción
            frame = add_emotion_effects(frame, emotion_idx, x, y, w, h, confidence)

        # Agregar información general en la parte superior
        if emotion_detected is not None:
            config = emotion_config[emotion_detected]
            info_text = f"Emocion Principal: {config['name']}"
            cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, config['color'], 2)

            # Cambiar el color de fondo del marco general según emoción
            overlay = frame.copy()
            color = config['color']
            cv2.rectangle(overlay, (0, 0), (frame.shape[1], 60), color, -1)
            cv2.addWeighted(overlay, 0.1, frame, 0.9, 0, frame)

        # Mostrar el frame
        cv2.imshow('Deteccion de Emociones', frame)

        # Presionar 'q' para salir
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_event.set()
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()
    print("Deteccion finalizada")

# Crear e iniciar el hilo
thread = threading.Thread(target=emotion_detection_thread, daemon=True)
thread.start()

# Esperar a que el hilo termine
thread.join(timeout=300)  # Timeout de 5 minutos

print("Sistema de detección de emociones finalizado")

Ventana abierta. Presiona 'q' para salir.
Las emociones cambiarán el color y los efectos visuales

Deteccion finalizada
Sistema de detección de emociones finalizado
